<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Running-this-notebook" data-toc-modified-id="Running-this-notebook-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Running this notebook</a></span><ul class="toc-item"><li><span><a href="#Instructions-for-opening-this-notebook-(tested-in-MacOS-with-Python-3.7)" data-toc-modified-id="Instructions-for-opening-this-notebook-(tested-in-MacOS-with-Python-3.7)-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Instructions for opening this notebook (tested in MacOS with Python 3.7)</a></span></li></ul></li><li><span><a href="#Start-the-HOList-server" data-toc-modified-id="Start-the-HOList-server-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Start the HOList server</a></span></li><li><span><a href="#Interacting-with-the-server:-big-picture" data-toc-modified-id="Interacting-with-the-server:-big-picture-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Interacting with the server: big picture</a></span><ul class="toc-item"><li><span><a href="#Theorem-proving" data-toc-modified-id="Theorem-proving-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Theorem proving</a></span></li><li><span><a href="#Workflow" data-toc-modified-id="Workflow-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Workflow</a></span></li><li><span><a href="#The-server-state" data-toc-modified-id="The-server-state-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>The server state</a></span></li></ul></li><li><span><a href="#A-quick-walkthrough-of-the-gRPC-API" data-toc-modified-id="A-quick-walkthrough-of-the-gRPC-API-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>A quick walkthrough of the gRPC API</a></span><ul class="toc-item"><li><span><a href="#ApplyTactic-request" data-toc-modified-id="ApplyTactic-request-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span><code>ApplyTactic</code> request</a></span></li><li><span><a href="#VerifyProof-request" data-toc-modified-id="VerifyProof-request-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span><code>VerifyProof</code> request</a></span></li><li><span><a href="#RegisterTheorem-request" data-toc-modified-id="RegisterTheorem-request-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span><code>RegisterTheorem</code> request</a></span></li></ul></li><li><span><a href="#Available-tactics" data-toc-modified-id="Available-tactics-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Available tactics</a></span></li><li><span><a href="#TODO:-Figure-out-how-all-this-fits-into-the-big-workflow-of-using-the-HOList-Dataset." data-toc-modified-id="TODO:-Figure-out-how-all-this-fits-into-the-big-workflow-of-using-the-HOList-Dataset.-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>TODO: Figure out how all this fits into the big workflow of using the HOList Dataset.</a></span></li><li><span><a href="#Don't-forget-to-shutdown-the-HOList-server-when-done" data-toc-modified-id="Don't-forget-to-shutdown-the-HOList-server-when-done-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Don't forget to shutdown the HOList server when done</a></span></li></ul></div>

## Running this notebook

### Instructions for opening this notebook (tested in MacOS with Python 3.7)
Clone repository
```bash 
$ git clone https://github.com/jasonrute/holist-communication-example
$ cd holist-communication-example
```

Make new virtual environment using Python's built in venv (virtualenv and conda should also work).
```bash
$ python3.7 -m venv venv  # make a new virtual env
$ source venv/bin/activate
$ python -m pip install --upgrade pip
```

Install GRPC and Jupyter.
```
python -m pip install -r requirements.txt
```

Run Jupyter
```
jupyter notebook
```

Open the `walkthrough_of_holist_api` and you should be all set.

In [2]:
import time

import grpc

# automatically generated protobuf and grpc python files
import proof_assistant_pb2
import proof_assistant_pb2_grpc

## Start the HOList server

In [3]:
PORT = 'localhost:2000'

You must have [Docker](https://www.docker.com) installed and running.  The following command starts the HOList server and exposes port 2000.

In [3]:
# shut down the server in case it is running so that we have a clean start
! docker stop holist && docker rm holist

Got permission denied while trying to connect to the Docker daemon socket at unix:///var/run/docker.sock: Post "http://%2Fvar%2Frun%2Fdocker.sock/v1.24/containers/holist/stop": dial unix /var/run/docker.sock: connect: permission denied


In [5]:
# run the docker container on the command line 
! sudo docker run -d -p 2000:2000 --name=holist gcr.io/deepmath/hol-light

[sudo] password for sean: 


**IMPORTANT: The docker container needs to be shut down when finished.**
To shut down, on the command line, run 
```bash
$ docker stop holist && docker rm holist
```
or in this notebook, run
```python
! docker stop holist && docker rm holist
```

Testing that the gRPC can communicate with the server

In [4]:
print("Attempting to communicate with HOList server via", PORT, "...")

time.sleep(10) # give the server time to start up

with grpc.insecure_channel(PORT) as channel:
    stub = proof_assistant_pb2_grpc.ProofAssistantServiceStub(channel)
    
    trivial_theorem = proof_assistant_pb2.Theorem(
        name="True",
        conclusion="(c (bool) T)",  # True
        training_split=proof_assistant_pb2.Theorem.Split.TESTING,
        tag=proof_assistant_pb2.Theorem.Tag.THEOREM,
    )
    request = proof_assistant_pb2.ApplyTacticRequest(goal=trivial_theorem, tactic="ITAUT")
    
    # will fail if can't connect to server
    response = stub.ApplyTactic(request)
    
    print("Success!")

Attempting to communicate with HOList server via localhost:2000 ...
Success!


## Interacting with the server: big picture

### Theorem proving
One can think of tactic-based theorem proving as a type of one-player board game, or as a puzzle like the Rubik's cube.  There is a starting state (e.g. a shuffled Rubik's cube), intermediate states (e.g. a partially solved Rubik's cube), a desired final state (a solved Rubik's cube), and a set of actions one can do each turn (e.g. move the Rubik's cube). In HOL Light, it is the same.  A state in this case is a list of *goals*, and the actions are called *tactics*.  However, unlike a Rubik's cube or a board game, the tactic actions can be quite complicated, even referring to previously proved theorems.

Hence the big picture of the HOList interface is that it provides 3 ways to call the server:

1. Apply a tactic to a goal.  This is basically performing one action in the attempt to prove a goal.
2. Verify proof.  This is verifying that a proof (a list of tactics) is correct.
3. Register a theorem (or definition).  This is the way to add dependency theorems that a tactic can refer to.  If I want to prove that `1 + 2 = 2 + 1`, I can use the theorem that addition is commutative.  To do that, I have to add this to the list of theorems that the server maintains.

Below we will go into details about all these server calls, but it is good to know that they are outlined in the file `proof_assistant.proto`, particularly, these lines: 
```proto
service ProofAssistantService {
  // Apply a tactic to a goal, potentially generating new subgoals.
  rpc ApplyTactic(ApplyTacticRequest) returns (ApplyTacticResponse) {}

  // Verify that a sequence of tactics proves a goal using the proof assistant.
  rpc VerifyProof(VerifyProofRequest) returns (VerifyProofResponse) {}

  // Register a new theorem with the proof assistant and verify/generate
  // its fingerprint. The theorem will be assumed as one that can be used
  // as a valid premise for later proofs.
  rpc RegisterTheorem(RegisterTheoremRequest)
      returns (RegisterTheoremResponse) {}
}
```

### Workflow
The intended workflow to use this server seems to be the following:
1. Compile a list of theorems along with their proofs.  (HOList provides this.)
2. In order, do the following for each theorem in the test set:
   1. Repeatedly use the `ApplyTactic` requests to try to prove a theorem, using backtracking.  Can refer to earlier added theorems and definitions.
   2. Use the `VerifyProof` request to verify that a proof is correct.  (This is probably not the proof one just found, but one supplied in the training data.)
   3. Use the `RegisterTheorem` request to add that theorem to the list of available theorems.  Note, it has to be the most recently verified proof for this to work.  Now this theorem is available as a premise for future theorems.

### The server state

It is important to observe that the HOList server is **stateful**.  The server maintains an internal list of theorems and definitions.  This list starts out empty, but can be added to later.  These theorems and definitions can be used as premises to prove other theorems.

It does not however store the current goal state.  You have to supply that in the server call.  This makes tree search and backtracking easy, since there is no concept of a "current goal state" in the server.

## A quick walkthrough of the gRPC API

### `ApplyTactic` request
The request is a combination of a goal and a tactic.  The type of the goal is a `Theorem` object and the type of the tactic is a plain string.  

The response is a list of new goals to solve.  If the list is empty, then that tactic solved that goal.

Theorem objects have a name, a conclusion (that which is being proved) and some more labels which have to be filled in.  The conclusion is a LISP-like s-expression based on the internal HOL Light language.  It is hard to read by a human.  Here is a quick tutorial:
- `(a F x)` means apply a function F to an object x (just written `F x` in functional programming).
- `(c T x)` is referring to the constant value x, which has type T (`x : T` in functional programming notation).
- `(v T x)` is similar, but for a variable x with type T.  
- `(fun A B)` is a function type constructor (`A -> B` in functional programming).  
- `(l v f)` is a lambda term (`lambda v. f`, or similar, in functional programming).
- `!` is "for all" which has type `(A -> bool) -> bool`
- `?` is "exists"
- `=` has type `A -> A -> bool`

In [5]:
# A basic theorem
eq_refl = proof_assistant_pb2.Theorem(
    name="EQ_REFL",
    conclusion="(a (c (fun (fun A (bool)) (bool)) !) (l (v A x) (a (a (c (fun A (fun A (bool))) =) (v A x)) (v A x))))",
    training_split=proof_assistant_pb2.Theorem.Split.TESTING,
    tag=proof_assistant_pb2.Theorem.Tag.THEOREM,
)

More on tactics later, but for now it is enough to know that a tactic is written as a string in a modified form of HOL Light's tactic DSL. Some tactics take no parameters, a single parameter, or a list of parameters.  `[ ]` means an empty list of parameters (spaces are important here).

In [6]:
# a simplying tactic with no parameters
simp_tac_no_params = 'SIMP_TAC [ ]'

In [7]:
# simplification is enough to prove reflection of equality
with grpc.insecure_channel(PORT) as channel:
    stub = proof_assistant_pb2_grpc.ProofAssistantServiceStub(channel)

    request = proof_assistant_pb2.ApplyTacticRequest(goal=eq_refl, tactic=simp_tac_no_params)
    print("Request:")
    print(request)

    response = stub.ApplyTactic(request)
    print("Response:")
    print(response)

Request:
goal {
  conclusion: "(a (c (fun (fun A (bool)) (bool)) !) (l (v A x) (a (a (c (fun A (fun A (bool))) =) (v A x)) (v A x))))"
  tag: THEOREM
  name: "EQ_REFL"
  training_split: TESTING
}
tactic: "SIMP_TAC [ ]"

Response:
goals {
}



A response of
```proto
goals { }
```
means the goal has been solved!

Let's try to prove something a bit more difficult.

In [8]:
for_all_x_exists_y_x_equals_y = proof_assistant_pb2.Theorem(
    name="FORALL_X_EXISTS_Y_SUCH_THAT_X_EQUALS_Y",
    conclusion="(a (c (fun (fun A (bool)) (bool)) !) (l (v A x) (a (c (fun (fun A (bool)) (bool)) ?) (l (v A y) (a (a (c (fun A (fun A (bool))) =) (v A x)) (v A y))))))",
    training_split=proof_assistant_pb2.Theorem.Split.TESTING,
    tag=proof_assistant_pb2.Theorem.Tag.THEOREM,
)

In [149]:
with grpc.insecure_channel(PORT) as channel:
    stub = proof_assistant_pb2_grpc.ProofAssistantServiceStub(channel)

    print("Remove the forall x")
    print()
    
    request1 = proof_assistant_pb2.ApplyTacticRequest(goal=for_all_x_exists_y_x_equals_y, tactic="GEN_TAC")
    print("Request:")
    print(request1)

    response1 = stub.ApplyTactic(request1)
    print("Response:")
    print(response1)
    
    print("Specify that exists y is satisfied by y = x")
    print()
    
    request2 = proof_assistant_pb2.ApplyTacticRequest(goal=response1.goals.goals[0], tactic="EXISTS_TAC `(v A x)`")
    print("Request:")
    print(request2)

    response2 = stub.ApplyTactic(request2)
    print("Response:")
    print(response2)
    
    print("Simplify to solve")
    print()
    
    request3 = proof_assistant_pb2.ApplyTacticRequest(goal=response2.goals.goals[0], tactic="SIMP_TAC [ ]")
    print("Request:")
    print(request3)

    response3 = stub.ApplyTactic(request3)
    print("Response:")
    print(response3)

Remove the forall x

Request:
goal {
  conclusion: "(a (c (fun (fun A (bool)) (bool)) !) (l (v A x) (a (c (fun (fun A (bool)) (bool)) ?) (l (v A y) (a (a (c (fun A (fun A (bool))) =) (v A x)) (v A y))))))"
  tag: THEOREM
  name: "FORALL_X_EXISTS_Y_SUCH_THAT_X_EQUALS_Y"
  training_split: TESTING
}
tactic: "GEN_TAC"

Response:
goals {
  goals {
    conclusion: "(a (c (fun (fun A (bool)) (bool)) ?) (l (v A y) (a (a (c (fun A (fun A (bool))) =) (v A x)) (v A y))))"
    tag: GOAL
    pretty_printed: "\n`?y. x = y`\n"
  }
}

Specify that exists y is satisfied by y = x

Request:
goal {
  conclusion: "(a (c (fun (fun A (bool)) (bool)) ?) (l (v A y) (a (a (c (fun A (fun A (bool))) =) (v A x)) (v A y))))"
  tag: GOAL
  pretty_printed: "\n`?y. x = y`\n"
}
tactic: "EXISTS_TAC `(v A x)`"

Response:
goals {
  goals {
    conclusion: "(a (a (c (fun A (fun A (bool))) =) (v A x)) (v A x))"
    tag: GOAL
    pretty_printed: "\n`x = x`\n"
  }
}

Simplify to solve

Request:
goal {
  conclusion: "(a (a (c 

In [10]:
response3

goals {
}

It is also possible to get back a failure as a response.  This happens if the tactic is not applicable to that goal, or if it just fails to make progress on the goal.

In [140]:
with grpc.insecure_channel(PORT) as channel:
    stub = proof_assistant_pb2_grpc.ProofAssistantServiceStub(channel)

    request = proof_assistant_pb2.ApplyTacticRequest(goal=for_all_x_exists_y_x_equals_y, tactic="EXISTS_TAC `(v A y)`")
    print("Request:")
    print(request)

    response = stub.ApplyTactic(request)
    print("Response:")
    print(response)

Request:
goal {
  conclusion: "(a (c (fun (fun A (bool)) (bool)) !) (l (v A x) (a (c (fun (fun A (bool)) (bool)) ?) (l (v A y) (a (a (c (fun A (fun A (bool))) =) (v A x)) (v A y))))))"
  tag: THEOREM
  name: "FORALL_X_EXISTS_Y_SUCH_THAT_X_EQUALS_Y"
  training_split: TESTING
}
tactic: "EXISTS_TAC `(v A y)`"

Response:
error: "Failure(\"EXISTS_TAC: Goal not existentially quantified\")"



### `VerifyProof` request

The request is a goal, a theorem (the same?), and a proof (as a list of strings of tactic commands).  The response is a boolean saying the proof has been verified (or an error).

Here we verify the theorem from earlier.

In [139]:
with grpc.insecure_channel(PORT) as channel:
    stub = proof_assistant_pb2_grpc.ProofAssistantServiceStub(channel)

    request = proof_assistant_pb2.VerifyProofRequest(
        goal=for_all_x_exists_y_x_equals_y, 
        theorem=for_all_x_exists_y_x_equals_y, 
        tactics=[
            "GEN_TAC",
            "EXISTS_TAC `(v A x)`",
            "SIMP_TAC [ ]",
        ]
    )
    
    print("Request:")
    print(request)

    response = stub.VerifyProof(request)
    print("Response:")
    print(response)

Request:
goal {
  conclusion: "(a (c (fun (fun A (bool)) (bool)) !) (l (v A x) (a (c (fun (fun A (bool)) (bool)) ?) (l (v A y) (a (a (c (fun A (fun A (bool))) =) (v A x)) (v A y))))))"
  tag: THEOREM
  name: "FORALL_X_EXISTS_Y_SUCH_THAT_X_EQUALS_Y"
  training_split: TESTING
}
tactics: "GEN_TAC"
tactics: "EXISTS_TAC `(v A x)`"
tactics: "SIMP_TAC [ ]"
theorem {
  conclusion: "(a (c (fun (fun A (bool)) (bool)) !) (l (v A x) (a (c (fun (fun A (bool)) (bool)) ?) (l (v A y) (a (a (c (fun A (fun A (bool))) =) (v A x)) (v A y))))))"
  tag: THEOREM
  name: "FORALL_X_EXISTS_Y_SUCH_THAT_X_EQUALS_Y"
  training_split: TESTING
}

Response:
sound: true



### `RegisterTheorem` request

The request is a theorem (or definition) to add.  The response is the fingerprint of the theorem.

**This theorem has to be the one from the most recent call of VerifyProof for this to work.**

In [13]:
with grpc.insecure_channel(PORT) as channel:
    stub = proof_assistant_pb2_grpc.ProofAssistantServiceStub(channel)

    request = proof_assistant_pb2.RegisterTheoremRequest(
        theorem=for_all_x_exists_y_x_equals_y
    )
    
    print("Request:")
    print(request)

    response = stub.RegisterTheorem(request)
    print("Response:")
    print(response)
    
    for_all_x_exists_y_x_equals_y_fingerprint = response.fingerprint

Request:
theorem {
  conclusion: "(a (c (fun (fun A (bool)) (bool)) !) (l (v A x) (a (c (fun (fun A (bool)) (bool)) ?) (l (v A y) (a (a (c (fun A (fun A (bool))) =) (v A x)) (v A y))))))"
  tag: THEOREM
  name: "FORALL_X_EXISTS_Y_SUCH_THAT_X_EQUALS_Y"
  training_split: TESTING
}

Response:
fingerprint: 2679395207268212325



This fingerprint can be used as a parameter for other tactic calls.

In [14]:
example1 = proof_assistant_pb2.Theorem(
    name="EXISTS_X_SUCH_THAT_TRUE_EQUALS_X",
    conclusion="(a (c (fun (fun (bool) (bool)) (bool)) ?) (l (v (bool) x) (a (a (c (fun (bool) (fun (bool) (bool))) =) (c (bool) T)) (v (bool) x))))",
    training_split=proof_assistant_pb2.Theorem.Split.TESTING,
    tag=proof_assistant_pb2.Theorem.Tag.THEOREM,
)

In [145]:
with grpc.insecure_channel(PORT) as channel:
    stub = proof_assistant_pb2_grpc.ProofAssistantServiceStub(channel)

    request = proof_assistant_pb2.ApplyTacticRequest(goal=example1, tactic="SIMP_TAC [ THM {} ]".format(for_all_x_exists_y_x_equals_y_fingerprint))
    print("Request:")
    print(request)

    response = stub.ApplyTactic(request)
    print("Response:")
    print(response)

Request:
goal {
  conclusion: "(a (c (fun (fun (bool) (bool)) (bool)) ?) (l (v (bool) x) (a (a (c (fun (bool) (fun (bool) (bool))) =) (c (bool) T)) (v (bool) x))))"
  tag: THEOREM
  name: "EXISTS_X_SUCH_THAT_TRUE_EQUALS_X"
  training_split: TESTING
}
tactic: "SIMP_TAC [ THM 2679395207268212325 ]"

Response:
goals {
}



Again, a theorem can't be registered unless it is the most recently verified proof.  **In some cases (such as if there are no registed proofs yet, the error message can be quite cryptic).**

In [16]:
with grpc.insecure_channel(PORT) as channel:
    stub = proof_assistant_pb2_grpc.ProofAssistantServiceStub(channel)

    request = proof_assistant_pb2.RegisterTheoremRequest(
        theorem=example1
    )
    
    print("Request:")
    print(request)

    response = stub.RegisterTheorem(request)
    print("Response:")
    print(response)

Request:
theorem {
  conclusion: "(a (c (fun (fun (bool) (bool)) (bool)) ?) (l (v (bool) x) (a (a (c (fun (bool) (fun (bool) (bool))) =) (c (bool) T)) (v (bool) x))))"
  tag: THEOREM
  name: "EXISTS_X_SUCH_THAT_TRUE_EQUALS_X"
  training_split: TESTING
}

Response:
fingerprint: 2193973276563729542



**TODO: Show how to register definitions**

## Available tactics

The tactic argument in Apply tactic is a modified form of HOL Light code.  Below is a list of tactics and how to call them.  See here for HOL Light documentation: https://www.cl.cam.ac.uk/~jrh13/hol-light/reference.html.

In [17]:
# tactics which take no arguements
abs_tac = "ABS_TAC"
ants_tac = "ANTS_TAC"
arith_tac = "ARITH_TAC"
cheat_tac = "CHEAT_TAC"  # NEVER USE EXCEPT FOR TESTING!
conj_tac = "CONJ_TAC"
disch_tac = "DISCH_TAC"
disj1_tac = "DISJ1_TAC"
disj2_tac = "DISJ2_TAC"
eq_tac = "EQ_TAC"
gen_tac = "GEN_TAC"
itaut_tac = "ITAUT_TAC"
mk_comb_tac = "MK_COMB_TAC"
raw_pop_all_tac = "RAW_POP_ALL_TAC"
real_arith_tac = "REAL_ARITH_TAC"  # Calc_rat.REAL_ARITH_TAC in HOL Light
real_arith_tac2 = "REAL_ARITH_TAC2"  # Reals.REAL_ARITH_TAC in HOL Light
refl_tac = "REFL_TAC"

# tactics which take a single theorem as a parameter
# The theorem is entered as "THM n" where n is the fingerprint of the theorem
accept_tac = lambda th: "ACCEPT_TAC THM {}".format(th)
backchain_tac = lambda th: "BACKCHAIN_TAC THM {}".format(th)
choose_tac = lambda th: "CHOOSE_TAC THM {}".format(th)
contr_tac = lambda th: "CONTR_TAC THM {}".format(th)
disj_cases_tac = lambda th: "DISJ_CASES_TAC THM {}".format(th)
match_accept_tac = lambda th: "MATCH_ACCEPT_TAC THM {}".format(th)
match_mp_tac = lambda th: "MATCH_MP_TAC THM {}".format(th)
mp_tac = lambda th: "MP_TAC THM {}".format(th)
raw_conjuncts_tac = lambda th: "RAW_CONJUNCTS_TAC THM {}".format(th)
subst1_tac = lambda th: "SUBST1_TAC THM {}".format(th)


# tactics which take a list of theorems (possibly empty) as a parameter
# The theorems are entered as a list "[ THM n1 ; THM n2 ; ... ]" 
# where n1, n2, ... are the fingerprints of the theorems
asm_meson_tac = lambda thms: "ASM_MESON_TAC [ {} ]".format(" ; ".join("THM {}".format(thm) for thm in thms))
asm_metis_tac = lambda thms: "ASM_METIS_TAC [ {} ]".format(" ; ".join("THM {}".format(thm) for thm in thms))
meson_tac = lambda thms: "MESON_TAC [ {} ]".format(" ; ".join("THM {}".format(thm) for thm in thms))
once_rewrite_tac = lambda thms: "ONCE_REWRITE_TAC [ {} ]".format(" ; ".join("THM {}".format(thm) for thm in thms))
pure_once_rewrite_tac = lambda thms: "PURE_ONCE_REWRITE_TAC [ {} ]".format(" ; ".join("THM {}".format(thm) for thm in thms))
pure_rewrite_tac = lambda thms: "PURE_REWRITE_TAC [ {} ]".format(" ; ".join("THM {}".format(thm) for thm in thms))
rewrite_tac = lambda thms: "REWRITE_TAC [ {} ]".format(" ; ".join("THM {}".format(thm) for thm in thms))
simp_tac = lambda thms: "SIMP_TAC [ {} ]".format(" ; ".join("THM {}".format(thm) for thm in thms))
                                  
# tactics which take a single term as a parameter
# the term is entered as an s-exp surronded by backticks (in the same syntax as terms in the the goal) 
# For example a boolean variable x would be entered as "`(v bool x)`"
exists_tac = lambda tm: "EXISTS_TAC `{}`".format(tm)
raw_subgoal_tac = lambda tm: "RAW_SUBGOAL_TAC `{}`".format(tm)
trans_tac = lambda tm: "TRANS_TAC `{}`".format(tm)
undisch_tac = lambda tm: "UNDISCH_TAC `{}`".format(tm)
x_gen_tac = lambda tm: "X_GEN_TAC `{}`".format(tm)
             
# tactics which take a conversion as a parameter (see below)
conv_tac = lambda conv: "CONV_TAC {}".format(conv)

# tactics which take a conversion function and a theorem list as parameters (see below)
gen_rewrite_tac = lambda convfn, thms: "GEN_REWRITE_TAC {} [ {} ]".format(convfn, " ; ".join("THM {}".format(thm) for thm in thms))
   
# tactics which take positive integers as parameters
raw_pop_tac = lambda n: "RAW_POP_TAC {}".format(n)
    
# tactics which take two terms (each s-exp in backquotes) as parameters
spec_tac = lambda tm1, tm2: "SPEC_TAC `{}` `{}`".format(tm1, tm2)
    
# tactics which take a term and a theorem as parameters
x_choose_tac = lambda tm, th: "X_CHOOSE_TAC `{}` THM th".format(tm, th)

For the `CONV_TAC` and `GEN_REWRITE_TAC` tactics one needs to supply a conversion or conversion function.  Here are the available conversions.

In [18]:
# HOL Light conversions
beta_conv = "BETA_CONV"
contrapos_conv = "CONTRAPOS_CONV"
gen_beta_conv = "GEN_BETA_CONV"
num_reduce_conv = "NUM_REDUCE_CONV"
real_rat_reduce_conv = "REAL_RAT_REDUCE_CONV"
sym_conv = "SYM_CONV"

# Apply a conversion function to a conversion
apply = lambda convfn, conv: "APPLY {} {}".format(convfn, conv)

# HOL Light conversion functions
binder_conv = "BINDER_CONV"
binop_conv = "BINOP_CONV"
land_conv = "LAND_CONV"
once_depth_conv = "ONCE_DEPTH_CONV"
rand_conv = "RAND_CONV"
rator_conv = "RATOR_CONV"
redepth_conv = "REDEPTH_CONV"
top_depth_conv = "TOP_DEPTH_CONV"

# Compose a list of conversion functions
compose = lambda convfnl: "COMPOSE [ {} ]".format(" ; ".join(convfnl))

## TODO: Figure out how all this fits into the big workflow of using the HOList Dataset.

## Don't forget to shutdown the HOList server when done

In [22]:
#! docker stop holist && docker rm holist

import google.protobuf.text_format as tf

thm_db = proof_assistant_pb2.TheoremDatabase()

# !pwd

with open("/home/sean/Documents/phd/holist/hollightdata/final/theorem_database_v1.1.textpb", "r") as f:
    tf.Parse(f.read(), thm_db)

thm_list = thm_db.theorems
thm_name = thm_db.name

thm_list[1]

thm_list[8]

len(thm_list)

31682

In [121]:
import io_util

import deephol_pb2

prf_log = io_util.read_protos("/home/sean/Documents/phd/holist/hollightdata/final/proofs/human/valid/test/prooflogs-00508-of-00528.pbtxts",deephol_pb2.ProofLog)

x = list(prf_log)

len(x)

2

In [158]:
prev = ""
seen = []
orig = x[0].nodes[0].goal.fingerprint
for node in x[0].nodes:
    y = (node.goal.fingerprint, node.proofs[0].tactic, [i.fingerprint for i in node.proofs[0].subgoals] if len(node.proofs[0].subgoals) > 0 else "done")
    print (y)
    
    if y[0] in seen:
        print ("dupe" + 20 * "!")
    else:
        seen.append(y[0])

    if y[1] == "LABEL_TAC":
        print (node)

(2952612442050326466, 'RAW_POP_ALL_TAC', [2952612442050326466])
(2952612442050326466, 'REWRITE_TAC', [3295225880719242845])
dupe!!!!!!!!!!!!!!!!!!!!
(3295225880719242845, 'REWRITE_TAC', 'done')


In [171]:
test_goal = x[0].nodes[0].goal

In [160]:
for node in x[0].nodes:
    print (5 * "\n")
    print (node.goal)
    print ("\n")
    print (node.proofs)







conclusion: "(a (a (c (fun (fun ?0 (bool)) (fun (fun ?0 (bool)) (bool))) =) (a (c (fun (fun ?0 (bool)) (fun ?0 (bool))) GSPEC) (l (v ?0 GEN%PVAR%0) (a (c (fun (fun ?0 (bool)) (bool)) ?) (l (v ?0 z) (a (a (a (c (fun ?0 (fun (bool) (fun ?0 (bool)))) SETSPEC) (v ?0 GEN%PVAR%0)) (a (a (c (fun (bool) (fun (bool) (bool))) /\\) (a (a (c (fun ?0 (fun (fun ?0 (bool)) (bool))) IN) (v ?0 z)) (v (fun ?0 (bool)) s))) (a (v (fun ?0 (bool)) P) (v ?0 z)))) (v ?0 z))))))) (a (a (c (fun (fun ?0 (bool)) (fun (fun ?0 (bool)) (fun ?0 (bool)))) INTER) (v (fun ?0 (bool)) s)) (a (c (fun (fun ?0 (bool)) (fun ?0 (bool))) GSPEC) (l (v ?0 GEN%PVAR%0) (a (c (fun (fun ?0 (bool)) (bool)) ?) (l (v ?0 z) (a (a (a (c (fun ?0 (fun (bool) (fun ?0 (bool)))) SETSPEC) (v ?0 GEN%PVAR%0)) (a (v (fun ?0 (bool)) P) (v ?0 z))) (v ?0 z))))))))"
tag: THEOREM
fingerprint: 2952612442050326466



[tactic: "RAW_POP_ALL_TAC"
subgoals {
  conclusion: "(a (a (c (fun (fun ?0 (bool)) (fun (fun ?0 (bool)) (bool))) =) (a (c (fun (fun ?

In [188]:
tac = x[0].nodes[1].proofs[0]
type(tac)

deephol_pb2.TacticApplication

In [ ]:
# idea : get negative (goal, tac) pairs from data when tac -> goal doesn't change anything, just run over large set of goals with all valid tactics and label all "Failures" with 0 

# idea : generate proof step by step using logs, and see if there's any unused goals for negative ([goal] -> selected_goal) samples
# need to convert from tac object to string, possible ignore some tacs e.g label tac 

# idea : for premise network, agent first decides on num_args, then that number is sampled from arg network

# idea : add premise pruning (given list of premises, try tactic application with subsets until same result isn't returned), do this through human dataset and RL data

# todo: figure out CONV and LABEL (replace with ASSUM_TAC?) ,RAW_POP_TAC

In [196]:
with grpc.insecure_channel(PORT) as channel:
    stub = proof_assistant_pb2_grpc.ProofAssistantServiceStub(channel)

    print("Remove the forall x")
    print()
    
    request1 = proof_assistant_pb2.ApplyTacticRequest(goal=test_goal, tactic="EQ_TAC")
    print("Request:")
    print(request1)

    response1 = stub.ApplyTactic(request1)
    print("Response:")
    print(response1)
    
    #check if duplicate conclusion 
#     print (response1.goals.goals[0].conclusion == test_goal.conclusion)
    
    
    request2 = proof_assistant_pb2.ApplyTacticRequest(goal=response1.goals.goals[0], tactic="RAW_POP_ALL_TAC")

    print()
    
#     request2 = proof_assistant_pb2.ApplyTacticRequest(goal=response1.goals.goals[0], tactic="EXISTS_TAC `(v A x)`")
#     print("Request:")
#     print(request2)

    response2 = stub.ApplyTactic(request2)
    print("Response:")
    print(response2)
    
#     print("Simplify to solve")
#     print()
    
#     request3 = proof_assistant_pb2.ApplyTacticRequest(goal=response2.goals.goals[0], tactic="SIMP_TAC [ ]")
#     print("Request:")
#     print(request3)

#     response3 = stub.ApplyTactic(request3)
#     print("Response:")
#     print(response3)

Remove the forall x

Request:
goal {
  conclusion: "(a (a (c (fun (fun ?0 (bool)) (fun (fun ?0 (bool)) (bool))) =) (a (c (fun (fun ?0 (bool)) (fun ?0 (bool))) GSPEC) (l (v ?0 GEN%PVAR%0) (a (c (fun (fun ?0 (bool)) (bool)) ?) (l (v ?0 z) (a (a (a (c (fun ?0 (fun (bool) (fun ?0 (bool)))) SETSPEC) (v ?0 GEN%PVAR%0)) (a (a (c (fun (bool) (fun (bool) (bool))) /\\) (a (a (c (fun ?0 (fun (fun ?0 (bool)) (bool))) IN) (v ?0 z)) (v (fun ?0 (bool)) s))) (a (v (fun ?0 (bool)) P) (v ?0 z)))) (v ?0 z))))))) (a (a (c (fun (fun ?0 (bool)) (fun (fun ?0 (bool)) (fun ?0 (bool)))) INTER) (v (fun ?0 (bool)) s)) (a (c (fun (fun ?0 (bool)) (fun ?0 (bool))) GSPEC) (l (v ?0 GEN%PVAR%0) (a (c (fun (fun ?0 (bool)) (bool)) ?) (l (v ?0 z) (a (a (a (c (fun ?0 (fun (bool) (fun ?0 (bool)))) SETSPEC) (v ?0 GEN%PVAR%0)) (a (v (fun ?0 (bool)) P) (v ?0 z))) (v ?0 z))))))))"
  tag: THEOREM
  fingerprint: 2952612442050326466
}
tactic: "EQ_TAC"

Response:
error: "Failure(\"EQ_TAC\")"



IndexError: list index (0) out of range

In [75]:
import ast_def

# !pip3 install numpy 

ModuleNotFoundError: No module named 'torch_geometric'

In [71]:
x[0].nodes[9].proofs[0].subgoals[0].conclusion

'(a (a (c (fun (bool) (fun (bool) (bool))) ==>) (a (a (c (fun (cart (real) (2)) (fun (fun (cart (real) (2)) (bool)) (bool))) IN) (v (cart (real) (2)) z)) (v (fun (cart (real) (2)) (bool)) s))) (a (c (fun (bool) (bool)) ~) (a (a (c (fun (cart (real) (2)) (fun (cart (real) (2)) (bool))) =) (a (a (c (fun (fun (cart (real) (2)) (cart (real) (2))) (fun (cart (real) (2)) (cart (real) (2)))) complex_derivative) (v (fun (cart (real) (2)) (cart (real) (2))) f)) (v (cart (real) (2)) z))) (a (c (fun (real) (cart (real) (2))) Cx) (a (c (fun (num) (real)) real_of_num) (a (c (fun (num) (num)) NUMERAL) (c (num) _0)))))))'

In [32]:
pf_log.theorem_in_database

import glob


files = glob.glob("/home/sean/Documents/phd/holist/hollightdata/final/proofs/human/**/*.pbtxt*")

len(files)

# 528 + 537 + 1664

count = 0
for i, file in enumerate(files):
    count += len(list(io_util.read_protos(file,deephol_pb2.ProofLog)))
    if i % 50 == 0:
        print (count)

 # 14742 proof logs total 

count

files

test = "(a (c (fun (fun (bool) (bool)) (bool)) ?asdf) (l (v (bool) xasdf) (a (a (c (fun (bool) (fun (bool) (bool))) =) (c yyy T)) (v (bool) x))))"

class Token:
    def __init__(self, value, type_):
        self.value = value
        self.type_ = type_
    
#def s_expression_to_token(exp):
    

class AST:
    def __init__(self, node, children=[], parent=None):
        self.node = node
        self.children = children
        self.parent = [parent]

    def _print(self, depth=1):
        print(depth * "--- " + self.node.value)
        if len(self.children) > 0:
            for child in self.children:
                child._print(depth + 1)

test

chars = list(test)

# helper which returns contents inside parentheses

def get_parentheses_val(exp):
    assert exp[0] == '('
    left_count = 1
    ret = ""
    while left_count > 0:
        exp.pop(0)
        if exp[0] == '(':
            left_count += 1
            ret += '('
        elif exp[0] == ')':
            left_count -= 1
            if left_count > 0:
                ret += ')'
            else:
                exp.pop(0)
        else:
            ret += exp[0]
    return ret

#helper to get arg up to next space
def get_arg(exp):
    assert exp[0] == " "
    exp.pop(0)

    if exp[0] == '(':
        val = get_parentheses_val(exp)
    else:
        val = ""
        assert exp[0] != " "
        while exp[0] != " " and exp[0] != ")":
            val += exp[0]
            exp.pop(0)
    return val

new_toks = []
while len(chars) > 0:
    if chars[0] == '(' or chars[0] == ")" or chars[0] == " ":
        new_toks.append(chars[0])
        chars.pop(0)
        
        
    elif chars[0] == 'a':        
        new_toks.append(chars[0])
        chars.pop(0)
    
    # either parentheses or space following char/var
    elif chars[0] == 'c' or chars[0] == 'v':
        
        func = chars[0]
      #  print (f"func {chars[0]}")
        chars.pop(0)
        
        val_type = get_arg(chars)
      #  print (f"type {val_type}")

        try:
            val_arg = get_arg(chars)
        except:
            print (chars)
        
        
#         print (f"arg {val_type, val_arg}")
        if func == 'c':
            new_toks.append(Token(val_type + " " + val_arg, "constant"))
        elif func == 'v':
            new_toks.append(Token(val_type + " " + val_arg, "variable"))
        else: 
            raise Error
        
                
    elif chars[0] == 'l':
        new_toks.append(chars[0])
        chars.pop(0)
    else:
        break


def get_parentheses_node(exp):
    assert exp[0] == '('
    left_count = 1
    ret = []
    while left_count > 0:
        exp.pop(0)
        if exp[0] == '(':
            left_count += 1
            ret.append('(')
        elif exp[0] == ')':
            left_count -= 1
            if left_count > 0:
                ret.append(')')
            else:
                exp.pop(0)
        else:
            ret.append(exp[0])
        
    print (ret)
    return ret

def get_arg_node(exp):
    if exp[0] == '(':
        print ("get parentheses")
        return get_arg_node(get_parentheses_node(exp))
    
    elif exp[0] == 'a':
        exp.pop(0)
        exp.pop(0)
        func = get_arg_node(exp) 
        exp.pop(0)
        val = get_arg_node(exp)
        new_node = AST(Token(val, type_ = "func"))
        func.children.append(new_node)
        print (f"a: val, func: {val.node.value, func.node.value}")
        return func
    
    elif exp[0] == 'l':
        assert exp[1] == " "
        exp.pop(0)
        exp.pop(0)
        var = get_arg_node(exp)
        exp.pop(0)
        func = get_arg_node(exp)
        lamb = AST(Token("|", "lambda"))
        lamb.children.append(var)
        lamb.children.append(func)
        print (f"lam: var, func: {var.node.value, func.node.value}")
        return lamb
    
    elif type(exp[0]) == Token:
        print (f"Tok: {exp}")
        ret = AST(exp[0])
        exp.pop(0)
        return ret

new_toks

ast = get_arg_node(new_toks)

ast.node.value#children

1
51
101
151
201
251


KeyboardInterrupt: 